In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon
from tqdm import tqdm
import gdown
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode
init_notebook_mode(connected= True)
import plotly.express as px

# import base ???

In [ ]:
base = pd.read_csv("INSEE_DATA_TREATED.csv", delimiter=",")

In [ ]:
base.columns

In [ ]:
numerical_columns = [
    'DATDEB', 'DATFIN', #date début et fin de rémunération par rapport au 01/01
     'AGE', #age en années
     'POND', #pondération 1/12e
    'DUREE', #durée de paie en jours
      'NBHEUR', 'NBHEUR_TOT', #nombre d'heures salariées total (quelle diff?)
    'WAGE', #transformation of TRNNETO
    'UNEMP' #transformation of TRALCHT
]
categorical_columns = [
    'A6', 'A17', 'A38' #activité en nomenclature agrégrée
    'CPFD', #temps complet ou partiel
    'DEPR', 'DEPT', #département résidence et travail
    'DOMEMPL', 'DOMEMPL_EM', #domaine de l'emploi et l'établissement d'affectation/employeur
    
    'FILT', #indic poste annexe 2 ou non-annexe 1 (seuils rémunération volume)
    'REGR', 'REGT', #région de résidence et de travail
    'SEXE', #1 homme 2 femme
    'PCS', #PCS-ESE
    'TYP_EMPLOI', #ordinaire, apprenti, autre
    'CONV_COLL', #convention collective
  
    'TRNNETO', #rémunération nette globale en tranches -> à passer en numérique ?
    'TRALCHT', #total des indémnités de chômage, en tranches -> passage en numérique ?
    'TREFF', #tranche d'effectif : de 0 à 250+ postes
    'CONT_TRAV', #contrat de travail : APP apprentissage, TOA occasionnel ou à l'acte, TTP intérim, AUTre
    'CS', #CSP mais code plus simple 
    'AGE_TR', #age en tranches quadriennales
    'DATDEB_TR',
       'DATFIN_TR', #dates début et fin rémunération en tranches
    'DUREE_TR', #durée de paie exprimée en jours en tranches mensuelles
]

### Exploratory data analysis: descriptive analysis

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['DATDEB'], bins=100, cumulative=False)
ax.set_xlabel('Date of start of revenue with respect to the 01/01')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['DATFIN'], bins=100, cumulative=False)
ax.set_xlabel('Date of end of revenue with respect to the 01/01')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['DATFIN'], bins=100, cumulative=False, log=True)
ax.set_xlabel('Date of end of revenue with respect to the 01/01')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['TRNNETO'], bins=100, cumulative=False)
ax.set_xlabel('Date of start of revenue with respect to the 01/01')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
base['TRNNETO'].value_counts()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['AGE'], bins=100, cumulative=False)
ax.set_xlabel('Age')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
categories = base['AGE_TR'].value_counts().index
counts = base['AGE_TR'].value_counts().values
fig, ax = plt.subplots()
ax.bar(categories, counts, width=0.5)
ax.set_xlabel("Age par tranche")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['CS'], bins=100, cumulative=False)
ax.set_xlabel('CSP simplifié')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
categories = base['CS_N'].value_counts().index
counts = base['CS_N'].value_counts().values
fig, ax = plt.subplots()
ax.barh(categories, counts)
ax.set_xlabel("Catégorie Socio-Professionnelle")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
categories = base['CONT_TRAV'].value_counts().index
counts = base['CONT_TRAV'].value_counts().values
fig, ax = plt.subplots()
ax.bar(categories, counts, width=0.5)
ax.set_xlabel('Contrat de travail')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
categories = base['DOMEMPL_EM_N'].value_counts().index
counts = base['DOMEMPL_EM_N'].value_counts().values
fig, ax = plt.subplots()
ax.barh(categories, counts)
ax.set_xlabel("Domaine d'emploi")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
categories = base['DOMEMPL'].value_counts().index
counts = base['DOMEMPL'].value_counts().values
fig, ax = plt.subplots()
ax.bar(categories, counts, width=0.5)
ax.set_xlabel("Domaine d'emploi")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['DUREE'], bins=100, cumulative=False)
ax.set_xlabel('Durée de paie, en jours')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['DUREE'], bins=100, cumulative=False, log=True)
ax.set_xlabel('Durée de paie, en jours')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
categories = base['REGT'].value_counts().index
counts = base['REGT'].value_counts().values
fig, ax = plt.subplots()
ax.bar(categories, counts, width=0.5)
ax.set_xlabel("Région de travail")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
categories = base['REGT_N'].value_counts().index
counts = base['REGT_N'].value_counts().values
fig, ax = plt.subplots()
ax.barh(categories, counts)
ax.set_xlabel("Région de travail")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
categories = base['REGR'].value_counts().index
counts = base['REGR'].value_counts().values
fig, ax = plt.subplots()
ax.bar(categories, counts, width=0.5)
ax.set_xlabel("Région de résidence")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
categories = base['REGR_N'].value_counts().index
counts = base['REGR_N'].value_counts().values
fig, ax = plt.subplots()
ax.barh(categories, counts)
ax.set_xlabel("Région de résidence")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['NBHEUR'], bins=100, cumulative=False)
ax.set_xlabel("Nombre d'heures salariées")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['POND'], bins=100, cumulative=False)
ax.set_xlabel("Nombre d'heures salariées")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
categories = base['DEPR_N'].value_counts().index
counts = base['DEPR_N'].value_counts().values
fig, ax = plt.subplots()
ax.barh(categories, counts)
ax.set_xlabel("Département de résidence")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['WAGE'], bins=100, cumulative=False)
ax.set_xlabel("Salaire")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['UNEMP'], bins=100, cumulative=False)
ax.set_xlabel("Chômage")
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
base['UNEMP'].value_counts()